In [3]:
!pip install transformers

In [4]:
!pip install -U Sagemaker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 43.2 MB/s eta 0:00:00
  Attempting uninstall: Sagemaker
    Found existing installation: sagemaker 2.244.0
    Uninstalling sagemaker-2.244.0:
      Successfully uninstalled sagemaker-2.244.0


In [9]:
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel

try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'cardiffnlp/twitter-roberta-base-sentiment-latest',
	'HF_TASK':'text-classification'
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	transformers_version='4.49.0',
	pytorch_version='2.6.0',
	py_version='py312',
	env=hub,
	role=role, 
)

In [10]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1, # number of instances
	instance_type='ml.m5.xlarge' # ec2 instance type
)

-----!

In [11]:
predictor.predict({
	"inputs": "I like you. I love you",
})

[{'label': 'positive', 'score': 0.9299362301826477}]

In [12]:
predictor.predict({
	"inputs": "omg! what a sad day",
})

[{'label': 'negative', 'score': 0.9263506531715393}]

In [16]:
predictor.predict({
	"inputs": "Today is friday",
})

[{'label': 'neutral', 'score': 0.7308969497680664}]